In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.data_sources.clob import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [ ]:
# Constants
CONNECTOR_NAME = "okx"
QUOTE_ASSET = "USDT"
INTERVAL = "1d"
MIN_NOTIONAL_SIZE = 5
DAYS = 360
FETCH_CANDLES = True
# Features configuration
VOLATILITY_WINDOW = 20
VOLUME_SHORT_WINDOW = 20
VOLUME_LONG_WINDOW = 50

# Download data
- Get trading rules
- Get candles for the last x days

In [ ]:
import asyncio

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset(QUOTE_ASSET)\
    .filter_by_min_notional_size(Decimal(MIN_NOTIONAL_SIZE))\
    .get_all_trading_pairs()

In [ ]:
trading_pairs

In [ ]:
BATCH_CANDLES_REQUEST = 2
SLEEP_REQUEST = 2.0


if FETCH_CANDLES:
    candles = await clob.get_candles_batch_last_days(CONNECTOR_NAME, trading_pairs, INTERVAL, DAYS, BATCH_CANDLES_REQUEST, SLEEP_REQUEST)
    clob.dump_candles_cache(root_path)
else:
    clob.load_candles_cache(root_path)

In [ ]:
candles = [value for key, value in clob.candles_cache.items() if key[2] == INTERVAL and key[0] == CONNECTOR_NAME]

In [ ]:
from core.features.candles.volatility import VolatilityConfig
from core.features.candles.volume import VolumeConfig
from research_notebooks.dneitor.utils import generate_report

report = generate_report(
    candles=candles,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW))
report

In [ ]:
VOLUME_QUANTILE = 0.5
NATR_QUANTILE = 0.5
CURRENT_POSITION_THRESHOLD = 0.8
TOP_X_MARKETS = 20

top_markets = report[(report['average_volume_per_hour'] > report['average_volume_per_hour'].quantile(VOLUME_QUANTILE)) &
                     (report['mean_natr'] > report['mean_natr'].quantile(NATR_QUANTILE)) &
                     (report['current_position'] > CURRENT_POSITION_THRESHOLD)]
top_markets = top_markets.sort_values(by='current_position', ascending=False).head(TOP_X_MARKETS)
top_markets_candles = {candle.trading_pair: candle for candle in candles if candle.trading_pair in top_markets["trading_pair"].values}



In [ ]:
top_markets_candles["DOGS-USDT"].plot()

In [ ]:
from hummingbot.core.data_type.common import OrderType
from research_notebooks.dneitor.utils import generate_config
from core.utils import dump_dict_to_yaml


TOTAL_AMOUNT = 500  # General total amount for all markets
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_OPEN_ORDERS = 1  # Input max open orders for each market
MIN_ORDER_AMOUNT = 5  # Minimum order amount for each market
LEVERAGE = 1  # Leverage for each market
TAKE_PROFIT_ORDER_TYPE = OrderType.MARKET

TAKE_PROFIT_MULTIPLIER = 0.2  # Multiplier for take profit based on NATR
AMOUNTS_QUOTE_PCT = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2]  # Weights for each cluster

strategy_config = generate_config(
    id=f"dneitor-{CONNECTOR_NAME}_0.6",
    connector_name=CONNECTOR_NAME,
    candles=top_markets_candles,
    top_markets=top_markets,
    total_amount=TOTAL_AMOUNT,
    amounts_quote_pct=AMOUNTS_QUOTE_PCT,
    activation_bounds=ACTIVATION_BOUNDS,
    take_profit_multiplier=TAKE_PROFIT_MULTIPLIER,
    max_open_orders=MAX_OPEN_ORDERS,
    min_order_amount=MIN_ORDER_AMOUNT,
    leverage=LEVERAGE,
    take_profit_order_type=TAKE_PROFIT_ORDER_TYPE
)

In [ ]:
strategy_config

In [ ]:
for config in strategy_config:
    dump_dict_to_yaml(config, "configs")